# Semestrální práce - Radomír Čech

Analýza dat sčítání lidu 2021

### Dostupnost dat

- jedná se o otevřená data, vysvětlení otevřených dat
- primárně jsou data rozdělená podle typů územních úrovní, některé více do podrobna, některé méně, témata (data) jsou ale stejná
- data jsou strukturovaná do CSV souborů s techinickými schématy v podobě JSON souborů, které obsahují vysvětlení atributů a jejich datové typy
- CSV formát je určité jednodušší pro práci s daty, ale vyžaduje stažení všech dat (oproti API, kde je moźné si stáhnout určitou část dat)
- není dostupný žádný náhled dat pro představu

### Struktura dat

- jedná se o CSV soubory, neboli textový soubor s hodnotami oddělenými oddělovačem, v tomto případě čárkou
- soubory jsou různé velikosti, občas desítky KB, občas desítky MB
- data lze otevřít v jakémkoli textovém editoru, ale je vhodné je převést do prezentovatelné podoby (např. tabulky)
- nejjednodušším nástrojem je pravděpodobně MS Excel
    - dokáže zobrazit náhled CSV souboru v původní textové podobě
    - pro převod do tabulkové podoby je nutné otevřít prázdný sešit a v kartě "Data" použít "Z textu/CSV"
    - někdy nutné nastavit kódování na UTF-8 a správný oddělovač (čárka)
    - v závislosti na velikosti datasetu se mohou data převádět i několik minut
    - výsledkem je kontigenční tabulka, která podporuje filtrování a jiné funkce
- pro programátorské řešení jsem využil databázový systém
    - pokud je třeba v datech vyhledávat programově, není CSV vhodný a efektivní formát, protože by vyžadoval sekvenční průchod všech dat
    - bylo rozhodnuto o využití NoSQL databázového systému MongoDB, NoSQL je vhodné díky rychlosti práce s daty + data neobsahují vzájemné relace
    - porovnání dat v CSV formátu a NoSQL formátu
        - příklad: obyvatelstvo podle pohlaví a pětiletých věkových skupin
            - velikost původních CSV dat: 65,5 MB
            - velikost kontigenční tabulky v MS Excel: 29 MB (o 56 % menší)
            - velikost MongoDB kolekce + index: 20,7 + 4,6 = 25,3 MB (o 62 % menší)

### Analýza a práce s daty

- pro jednodušší operace je vhodné používat kontigenční tabulku v MS Excel
    - např. zjištění nejlidnatějšího kraje
- pro náročnější operace je vhodné používat programátorské řešení přes MongoDB
    - např. ...
    - pro procházení dat v databázi se doporučuje program přímo od tvůrců MongoDB - MongoDB Compass, který je dostupný na Windows, MacOS a Linux
- atributy v každém datasetu se mohou mírně lišit, zde je příklad atributů v datasetu "Obyvatelstvo podle věku a pohlaví":

    | SLOUPEC     | VÝZNAM                                                                                                                   |
    |-------------|--------------------------------------------------------------------------------------------------------------------------|
    | idhod       | unikátní   identifikátor údaje Veřejné databáze ČSÚ                                                                      |
    | hodnota     | zjištěná   hodnota                                                                                                       |
    | ukaz_kod    | kód   ukazatele (statistické proměnné)                                                                                   |
    | vek_cis     | číselník   pro věkovou skupinu; pokud není uveden, jedná se o celkový údaj bez rozlišení   věkové skupiny                |
    | vek_kod     | kód   položky z číselníku pro věkovou skupinu; pokud není uveden, jedná se o   celkový údaj bez rozlišení věkové skupiny |
    | pohlavi_cis | číselník   pro pohlaví; pokud není uveden, jedná se o celkový údaj bez rozlišení pohlaví                                 |
    | pohlavi_kod | kód   položky z číselníku pro pohlaví; pokud není uveden, jedná se o celkový údaj   bez rozlišení pohlaví                |
    | uzemi_cis   | kód   číselníku pro území                                                                                                |
    | uzemi_kod   | kód   položky číselníku pro území                                                                                        |
    | sldb_rok    | rok   sčítání ve formátu YYYY                                                                                            |
    | sldb_datum  | rozhodné   datum sčítání ve formátu YYYY-MM-DD                                                                           |
    | ukaz_txt    | text   ukazatele                                                                                                         |
    | vek_txt     | text   položky z číselníku pro věkovou skupinu                                                                           |
    | pohlavi_txt | text   položky z číselníku pro pohlaví                                                                                   |
    | uzemi_txt   | text   položky z číselníku pro území                                                                                     |

<br>

- problematické mohou být atributy jako `vek_kod` nebo `uzemi_cis`, tyto atributy obsahují CIS hodnoty reprezentující hodnotu z daného číselníku
    - např. `uzemi_cis` popisuje územní úroveň dané hodnoty (třeba okres, kraj...), takže pokud chceme pracovat třeba s daty okresů, musíme znát tyto hodnoty a podle nich filtrovat
    - některé z těchto atributů mají v datech i svůj textový popis, např. `uzemi_kod`, který má hodnotu '500011', má svůj slovný popis v atributu `uzemi_txt` s hodnotou 'Želechovice nad Dřevnicí'
    - některé atributy nemají žádný textový popis, např. `uzemi_cis`
    - příklad hodnot pro `uzemi_cis` a jejich vysvětlení:

        | KÓD | NÁZEV                                       |
        |-----|---------------------------------------------|
        | 97  | Stát                                        |
        | 99  | Region soudržnosti                          |
        | 100 | Kraj                                        |
        | 101 | Okres                                       |
        | 65  | Správní obvody obcí s rozšířenou působností |
        | 72  | Správní obvod v hlavním městě Praze         |
        | 43  | Obec a vojenský újezd                       |
        | 44  | Městský obvod, městská část                 |

<br>

- každý záznam obsahuje také své vlastní unikátní ID, to ale není nijak využitelné v našem případě
- některé záznamy obsahují prázdné hodnoty, to má svůj význam, např. když atribut `pohlavi_kod` má prázndnou hodnotu, znamená to, že tento záznam uchovává hodnotu pro všechna pohlaví (hodnota za muže a ženy dohromady), to samé platí i např. pro `vek_kod`
- některé atributy jsou trochu matoucí, atributy `ukaz_kod`, `vek_cis`, `pohlavi_cis` jsou vždy stejné, tudíž nepředstavují žádný význam pro tento dataset ("Obyvatelstvo podle věku a pohlaví"), navíc nejsou čitelné, protože se jedná o kódy, to se ale může lišit v jiných datasetech
- také se opakuje atribut `sldb_rok`, který představuje rok sběru dat, ačkoli má tento atribut také pouze jednu hodnotu, může aspoň připomenout uživateli o jaký rok se jedná
- nejdůležitějším atributem je atribut `hodnota`, který obsahuje hodnotu daného záznamu (např. počet lidí s danou vlastností)

### Analýza datasetu "Obyvatelstvo podle pětiletých věkových skupin a pohlaví"

##### Základní data

- celkový počet obyvatel: 10,524,167
    - celkový počet žen: 5,337,619 (50,72 % z celkového počtu)
    - celkový počet mužů: 5,168,548 (49,28 % z celkového počtu)
- celkový počet obyvatel podle věkových fází a pohlaví:
    - děti (0-9 let)
        - celkem: 1,110,656 (10,55 % z celkového počtu)
            - ženy: 541,695 (48,7 % z celkových dětí)
            - muži: 568,961 (51,3 % z celkových dětí)
    - dospívání (10-19 let)
        - celkem: 1,070,940 (10,18 % z celkového počtu)
            - ženy: 522,033 (48,7 % z celkového počtu dospívajících)
            - muži: 548,907 (51,3 % z celkového počtu dospívajících)
    - dospělí (20-59 let)
        - celkem: 5,577,915 (53 % z celkového počtu)
            - ženy: 2,713,576 (48,6 % z celkového počtu dospělých)
            - muži: 2,864,339 (51,4 % z celkového počtu dospělých)
    - staří (60 a více)
        - celkem: 2,764,656 (26,27 % z celkového počtu)
            - ženy: 1,560,315 (56,4 % z celkového počtu starších)
            - muži: 1,204,341 (43,6 % z celkového počtu starších)

- poměr počtu obyvatel v pozdním věku podle pohlaví a věku:
    - 50 - 59 let:
        - ženy: 668,842 (49,4 %)
        - muži: 685,659 (50,6 %)
    - 60 - 69 let:
        - ženy: 672,176 (52,3 %)
        - muži: 612,513 (47,7 %)
    - 70 - 79 let:
        - ženy: 595,034 (57,3 %)
        - muži: 442,963 (42,7 %)
    - 80 - 89 let:
        - ženy: 245,967 (64,9 %)
        - muži: 132,717 (35,1 %)
    - 90 - 99 let:
        - ženy: 46,602 (74,4 %)
        - muži: 16,037 (25,6 %)
    - 100 a více let
        - ženy: 536 (82,8 %)
        - muži: 111 (17,2 %)

- ačkoli je celkový počet žen větší než počet mužů, počet mužů do věku cca 60 je stále vyšší než počet žen, od pozdního věku (60+) je počet žen drasticky vyšší a lze spatřit, že se ženy dožívají vyššího věku, ve věku 90+ je žen téměř 3x víc než mužů